In [128]:
import html
import re
from html.parser import HTMLParser

In [129]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
import sklearn
from sklearn.externals import joblib



In [130]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 

In [108]:
class ReutersParser(HTMLParser):
    """
    ReutersParser subclasses HTMLParser and is used to open the SGML
    files associated with the Reuters-21578 categorised test collection.

    The parser is a generator and will yield a single document at a time.
    Since the data will be chunked on parsing, it is necessary to keep 
    some internal state of when tags have been "entered" and "exited".
    Hence the in_body, in_topics and in_topic_d boolean members.
    """
    def __init__(self, encoding='latin-1'):
        """
        Initialise the superclass (HTMLParser) and reset the parser.
        Sets the encoding of the SGML files by default to latin-1.
        """
        html.parser.HTMLParser.__init__(self)
        self._reset()
        self.encoding = encoding

    def _reset(self):
        """
        This is called only on initialisation of the parser class
        and when a new topic-body tuple has been generated. It
        resets all off the state so that a new tuple can be subsequently
        generated.
        """
        self.in_body = False
        self.in_topics = False
        self.in_topic_d = False
        self.body = ""
        self.topics = []
        self.topic_d = ""

    def parse(self, fd):
        """
        parse accepts a file descriptor and loads the data in chunks
        in order to minimise memory usage. It then yields new documents
        as they are parsed.
        """
        self.docs = []
        for chunk in fd:
            self.feed(chunk.decode(self.encoding))
            for doc in self.docs:
                yield doc
            self.docs = []
        self.close()

    def handle_starttag(self, tag, attrs):
        """
        This method is used to determine what to do when the parser
        comes across a particular tag of type "tag". In this instance
        we simply set the internal state booleans to True if that particular
        tag has been found.
        """
        if tag == "reuters":
            pass
        elif tag == "body" or tag=="TITLE":
            self.in_body = True
        elif tag == "topics":
            self.in_topics = True
        elif tag == "d":
            self.in_topic_d = True 

    def handle_endtag(self, tag):
        """
        This method is used to determine what to do when the parser
        finishes with a particular tag of type "tag". 

        If the tag is a  tag, then we remove all 
        white-space with a regular expression and then append the 
        topic-body tuple.

        If the tag is a  or  tag then we simply set
        the internal state to False for these booleans, respectively.

        If the tag is a  tag (found within a  tag), then we
        append the particular topic to the "topics" list and 
        finally reset it.
        """
        if tag == "reuters":
            self.body = re.sub(r'\s+', r' ', self.body)
            self.docs.append( (self.topics, self.body) )
            self._reset()
        elif tag == "body" or tag=="TITLE":
            self.in_body = False
        elif tag == "topics":
            self.in_topics = False
        elif tag == "d":
            self.in_topic_d = False
            self.topics.append(self.topic_d)
            self.topic_d = ""  

    def handle_data(self, data):
        """
        The data is simply appended to the appropriate member state
        for that particular tag, up until the end closing tag appears.
        """
        if self.in_body:
            self.body += data
        elif self.in_topic_d:
            self.topic_d += data

In [109]:
def obtain_topic_tags():
    """
    Open the topic list file and import all of the topic names
    taking care to strip the trailing "\n" from each word.
    """
    topics = open(
        "reuters/all-topics-strings.lc.txt", "r"
    ).readlines()
    topics = [t.strip() for t in topics]
    return topics

In [110]:
def filter_doc_list_through_topics(topics, docs):
    """
    Reads all of the documents and creates a new list of two-tuples
    that contain a single feature entry and the body text, instead of
    a list of topics. It removes all geographic features and only 
    retains those documents which have at least one non-geographic
    topic.
    """
    ref_docs = []
    for d in docs:
        if d[0] == [] or d[0] == "":
            continue
        for t in d[0]:
            if t in topics:
                d_tup = (t, d[1])
                ref_docs.append(d_tup)
                break
    return ref_docs

In [111]:
def create_tfidf_training_data(docs):
    """
    Creates a document corpus list (by stripping out the
    class labels), then applies the TF-IDF transform to this
    list. 

    The function returns both the class label vector (y) and 
    the corpus token/feature matrix (X).
    """
    # Create the training data class labels
    y = [d[0] for d in docs]
    
    # Create the document corpus list
    corpus = [d[1] for d in docs]

    # Create the TF-IDF vectoriser and transform the corpus
    vectorizer = TfidfVectorizer(min_df=1)
    X = vectorizer.fit_transform(corpus)
    joblib.dump(vectorizer, 'tfidf.pkl') 
    return X, y

In [112]:

def get_raw_data():
    """
    This function is used to parse through the entire .sgm files and obtain the dataset as a list
    """
    files = ["reuters/reut2-%03d.sgm" % r for r in range(0, 22)]
    parser = ReutersParser()

    # Parse the document and force all generated docs into
    # a list so that it can be printed out to the console
    docs = []
    for fn in files:
        for d in parser.parse(open(fn, 'rb')):
            docs.append(d)

    # Obtain the topic tags and filter docs through it 
    topics = obtain_topic_tags()
    ref_docs = filter_doc_list_through_topics(topics, docs)
    return ref_docs

In [113]:
def preprocess_data(raw_data):
    """
    This function removes all the stopwords and applies stemming on the sentences in the dataset and returns the vectorised form of
    data after applying tfidfvectoriser
    """
    temp=[]
    stop_words = set(stopwords.words('english')) 
    for i in range(len(raw_data)):
        tokens=word_tokenize(raw_data[i][1])
        ps = PorterStemmer() 
        filtered_sentence = tuple(ps.stem(w) for w in tokens if not w in stop_words)
        #joblib.
        temp.append((raw_data[i][0]," ".join(filtered_sentence)))
    return create_tfidf_training_data(temp)
    
    

In [114]:
def preprocess_test_text(data):
    """
    This function is used to preprocess the given input text document and returns vectorised form of the given text data .
    """
    from nltk import  sent_tokenize
    data = sent_tokenize(data)
    vect = joblib.load('tfidf.pkl')
    t = vect.transform(data)
    return t

In [115]:
def predict_data(data,model):
    """
    This function predicts the class of input text document (in the form of string). This takens in the vectorised form of input 
    data obtained as output of preprocess_test_text function and returns the predicted class.
    """
    return model.predict(data)

In [116]:
def prepare_raw_data():
    """
    This function returns the train and test split of dataset used for training the model
    """
    raw_data=get_raw_data()
    X,y=preprocess_data(raw_data)
    return train_test_split(
        X, y, test_size=0.2, random_state=42
    )

In [117]:

# Create the training-test split of the data
X_train, X_test, y_train, y_test =prepare_raw_data()

In [118]:
def train_model():
    """
    This function trains the Stochaistic Gradient Descent Classifier with the X_train and y_train data( training data set) and returns the model
    """
    from sklearn import linear_model
    clf = linear_model.SGDClassifier(max_iter=10000,tol=1e-3)
    model=clf.fit(X_train, y_train)
    return model



In [119]:
def get_accuracy_score():
    """
    This function gets the accuracy score of the model by validating the model with test data set
    """
    pred=predict_data(X_test,model)
    return sklearn.metrics.accuracy_score(y_test,pred)
    

In [120]:
model=train_model()

In [121]:
raw_data[0]

('cocoa',
 'Showers continued throughout the week in the Bahia cocoa zone, alleviating the drought since early January and improving prospects for the coming temporao, although normal humidity levels have not been restored, Comissaria Smith said in its weekly review. The dry period means the temporao will be late this year. Arrivals for the week ended February 22 were 155,221 bags of 60 kilos making a cumulative total for the season of 5.93 mln against 5.81 at the same stage last year. Again it seems that cocoa delivered earlier on consignment was included in the arrivals figures. Comissaria Smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end. With total Bahia crop estimates around 6.4 mln bags and sales standing at almost 6.2 mln there are a few hundred thousand bags still in the hands of farmers, middlemen, exporters and processors. There are doubts as to how much of this cocoa would be fit for export as s

In [122]:
text=raw_data[0][1]

In [123]:
predict_data(preprocess_test_text(text),model)

array(['cocoa', 'cocoa', 'grain', 'cocoa', 'cocoa', 'grain', 'cocoa',
       'cocoa', 'trade', 'grain', 'grain', 'grain', 'crude', 'earn',
       'trade', 'earn', 'grain', 'earn', 'grain', 'cocoa', 'acq'],
      dtype='<U15')

In [127]:
print("Accuracy score :",get_accuracy_score()*100,"%")

Accuracy score : 85.26824978012313 %
